In [2]:
import os
import pandas as pd
%matplotlib inline
import nltk
nltk.download('vader_lexicon')
from iexfinance.stocks import Stock
from datetime import date, datetime, timedelta

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/SashaG/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [6]:
api_key = os.getenv("news_api")
aapl = Stock("AAPL", token="sk_04ffc7f6d9de47e6adc680c4aa22b44d")

In [4]:
from newsapi import NewsApiClient
newsapi = NewsApiClient(api_key=api_key)

In [5]:
aapl_headlines = newsapi.get_everything(
    q="Apple OR Tim Cook OR AAPL",
    language="en",
    page_size=100,
    sort_by="relevancy",
)

In [6]:
print(f"Total articles about Apple: {aapl_headlines['totalResults']}")

Total articles about Apple: 68600


In [7]:
aapl_sentiments = []

for article in aapl_headlines["articles"]:
    try:
        sentiment = analyzer.polarity_scores(article["content"])
      
        aapl_sentiments.append({
            "text": article["content"],
            "date": article["publishedAt"][:10],
            "compound": sentiment["compound"],
            "positive": sentiment["pos"],
            "negative": sentiment["neg"],
            "neutral": sentiment["neu"]
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
aapl_df = pd.DataFrame(aapl_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
aapl_df = aapl_df[cols]

aapl_df.head()


,date,text,compound,positive,negative,neutral
0,2019-10-31,Apple CFO Luca Maestri has also announced that...,0.0000,0.000,0.000,1.000
1,2019-11-11,Apple managers say AR glasses could supplant t...,0.0000,0.000,0.000,1.000
2,2019-10-28,"Well, that was unexpected. With absolutely no ...",-0.6115,0.044,0.147,0.809
3,2019-10-25,Photo by Mark Wilson/Getty Images President Do...,0.4939,0.084,0.000,0.916
4,2019-10-11,Apple has received a fair amount of backlash o...,-0.4939,0.045,0.108,0.846


In [8]:
aapl_df.describe()

,compound,positive,negative,neutral
count,97.000000,97.000000,97.000000,97.000000
mean,0.130899,0.062876,0.039495,0.897639
std,0.465304,0.057970,0.055218,0.070269
min,-0.865800,0.000000,0.000000,0.739000
25%,-0.139500,0.000000,0.000000,0.852000
50%,0.102700,0.058000,0.000000,0.895000
75%,0.510600,0.107000,0.070000,0.951000
max,0.932000,0.254000,0.221000,1.000000


In [9]:
current_date = date.today()
past_date = date.today() - timedelta(weeks=4)

In [7]:
aapl_returns = pd.DataFrame(aapl.get_historical_prices())[
    ["date", "changePercent"]
].set_index("date")

,changePercent
date,
2019-10-09,0.0000
2019-10-10,1.3478
2019-10-11,2.6598
2019-10-14,-0.1439
2019-10-15,-0.2332
